

    ~copy the initial database, DON"T MODIFY DIRECTLY.
    *Delete videos that are not available.


    *Process videos by cropping them based on the timestamps and rename in the the following format
        ~Label-videoID-1
        if there are more than 1 videos, 
        Label-videoID-2, 3, etc;


    *Crop the width and the height of the videos based of the box coordinates provided in the dataset.





In [5]:
import cv2
import numpy as np
from numpy import asarray
import pandas as pd
from PIL import Image
import os
import math

Here is the format of the videos  I am treating
{"org_text": "emotional", "clean_text": "emotional", "start_time": 0.0, "signer_id": 20, "signer": 40, "start": 0, "end": 30, "file": "SignSchool Emotional", "label": 907, "height": 360.0, "fps": 23.976, "end_time": 1.251, "url": "www.youtube.com/watch?v=C59jcSo4fEI", "text": "emotional", "box": [0.059554219245910645, 0.2810196578502655, 1.0, 0.8543376922607422], "width": 640.0},

{<given-word>, <start time> <>}

importing the data


In [6]:
# test_df = pd.read_json("Datasets/MS-ASL/MS-ASL-with-new-labels/NEW-MSASL-train-file.json")

changing videos to numpy array (function):

https://github.com/dxli94/WLASL/blob/master/code/I3D/datasets/nslt_dataset.py

In [6]:
# changing an image into an array
def change_img(image_name):
    img = Image.open(f"{image_name}")
 
    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(img)
    
    # <class 'numpy.ndarray'>
    print(type(numpydata))
    
    #  shape
    print(numpydata.shape)


# change_img("test_frames/frame_330.jpg")

cutting the videos


splitting the videos into frames.

https://techtutorialsx.com/2021/04/29/python-opencv-splitting-video-frames/

In [8]:
# def load_rgb_frames_from_video(vid_root, vid, start, num, resize=(256, 256)):
#     video_path = os.path.join(vid_root, vid + '.mp4')

#     vidcap = cv2.VideoCapture(video_path)

#     frames = []

#     total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

#     vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
#     for offset in range(min(num, int(total_frames - start))):
#         success, img = vidcap.read()

#         w, h, c = img.shape
#         if w < 226 or h < 226:
#             d = 226. - min(w, h)
#             sc = 1 + d / min(w, h)
#             img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

#         if w > 256 or h > 256:
#             img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

#         img = (img / 255.) * 2 - 1

#         frames.append(img)

#     return np.asarray(frames, dtype=np.float32)

# #num is the number of frames
# li = load_rgb_frames_from_video("IMAGE_vision", "test-video.mp4", 0, 60)
# for i in li:
#     print(i)
#     print("hi")
#     type(li)

In [7]:
def get_image(video_url, video_path, start_time, end_time, fps):
    #get the right pathS
    capture = cv2.VideoCapture(f"training_video_data/{video_url}.mp4")


    frameNr = 0
    # fps=int(capture.get(cv2.CAP_PROP_FRAME_COUNTCAP))

    #time in seconds
    # start_time=start_time
    # end_time=end_time

    #size of the image
    # frameWidth = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frameHeight = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # frameWidth = width
    # frameHeight = height
    
    # frameCount = (end_time-start_time)

    start_frame=start_time*fps
    end_frame=end_time*fps

    # buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

    frames=[]
    while (True):
        success, img = capture.read()
        # print(frameNr)
        if success and frameNr>start_frame and frameNr<end_frame:
            if frameNr%30==0: 
                #cropping the image
             


                # #transform into numpy array
                w, h, c = img.shape

                if w < 226 or h < 226:
                    d = 226. - min(w, h)
                    sc = 1 + d / min(w, h)
                    img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

                if w > 256 or h > 256:
                    img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

                img = (img / 255.) * 2 - 1

                frames.append(img)

                #this line used to write it into a folder
                # cv2.imwrite(f'{video_url}/frame_{frameNr}.jpg', img)

        #end of the video, or end of the portion of the video
        if frameNr>end_frame:
            break
    
        frameNr = frameNr+1

    capture.release()


    #return a numpy array.
        #saves the video in label/url.npy
    mode=0o666

    
    try:
        #if file doesn't exist, create it   
        if os.path.isdir(f"train_data/{video_path}") == False:
            os.mkdir(f"train_data/{video_path}", mode)
        np.save(f"train_data/{video_path}/{video_url}", np.asarray(frames, dtype=np.float32), allow_pickle=True, fix_imports=True)
    
    except FileNotFoundError:
        pass

    return np.asarray(frames, dtype=np.float32)
        



#just give me the video path in here
# get_image(" ALY PUT THE VIDEO PATH, RUN SOME LOOP OR SOMETHING, and it should be fine.", "VIDEO URL (which will be the name of the folder)", "start_time", "endtime")

# get_image("alkfjdsl2", "test-video2", 5, 30, 29.97)

creating the folders for the labels


In [8]:
test_df = pd.read_json('NEW-MSASL_train.json')
test_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,available
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.0575446188, 0.21637457610000002, 1, 0.73008...",640,NaN,True
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794178, 0.16717177630000002, 0.93926274...",480,NaN,True
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594580000002, 0.32334136960000004, 1, ...",640,NaN,False
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.055698871600000005, 0.2517327964, 0.9968159...",480,NaN,True
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.0390439928, 0.2419818342, 1, 0.8586810827]",640,NaN,True


In [9]:
hi="hello/hi"
str(hi)
hi=hi.replace("/", " ")
print(hi)
hi=hi[:-1]
print(hi)

hello hi
hello h


In [10]:
for idx in range(2700, len(test_df)-1):
    #if the file is available:
    if test_df.loc[idx, "available"]==False:
        continue

    #file-name
    file_name=test_df.loc[idx, "url"][-11:]#[24:]
    # org_text=org_text.replace("/", " ")


    start_time = test_df.loc[idx, "start_time"]

    end_time=test_df.loc[idx, "end_time"]

    fps=test_df.loc[idx, "fps"]

    clean_text=test_df.loc[idx, "clean_text"]

    
    #replacing all odd characters
    str(clean_text)
    clean_text=clean_text.replace("/", " ")
    if clean_text[-1]==" ":
        clean_text[:-1]

    if idx%10==0:
        print(idx)

     #if file was already created:
    if os.path.exists(f"train_data/{clean_text}/{file_name}.npy") == True:
        continue

    get_image(file_name, clean_text, start_time, end_time, fps)
    

    

2700
2710
2720
2730
2740
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2880
2890
2900
2910
2920
2930
2940
2960
2970
2980
2990
3000
3170
3180
3190
3200
3210
3220
3230
3240
3260
3270
3280
3290
3310
3320
3330
3340
3350
3390
3400
3410
3420
3430
3440
3450
3460
3470
3480
3490
3500
3510
3520
3530
3540
3550
3560
3580
3590
3600
3610
3660
3670
3680
3690
3700
3710
3720
3730
3740
3750
3760
3770
3780
3790
3810
3820
3830
3840
3850
3860
3870
3880
3890
3900
3920
3930
3940
3950
3960
3970
3980
3990
4000
4010
4020
4030
4040
4050
4060
4070
4080
4090
4100
4110
4120
4130
4180
4190
4200
4210
4220
4240
4250
4260
4290
4300
4310
4320
4330
4340
4350
4360
4370
4400
4420
4430
4440
4450
4460
4470
4480
4490
4500
4510
4520
4530
4600
4610
4620
4640
4650
4660
4700
4710
4720
4730
4740
4750
4760
4770
4780
4790
4800
4810
4820
4830
4840
4850
4860
4870
4880
4890
4900
4910
4920
4930
4940
4950
4960
4970
4980
4990
5000
5010
5020
5030
5040
5050
5060
5070
5080
5090
5100
5110
5120
5130
5140
5150
5160
5180
5190
5330


In [3]:
src = cv2.cuda_GpuMat()
src

< cv2.cuda.GpuMat 0000025858C08EB0>